https://keras.io/preprocessing/image/

# TensorFlow 2.0

In [1]:
import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets 
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/Users/afraca/Documents/deep-base/venv/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/afraca/Documents/deep-base/venv/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/afraca/Documents/deep-base/venv/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/afraca

## Hyperparameter Tunning

In [2]:
num_epochs = 10
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.7

input_shape = (32, 32, 3)
num_classes = 10

## Preprocess

In [3]:
import pandas as pd

In [11]:
train_df = pd.read_csv('train_dataset.csv')

test_df = pd.read_csv('test_dataset.csv')
train_df.head()



,name,class
0,dataset/cifar/train/32270_deer.png,deer
1,dataset/cifar/train/21851_cat.png,cat
2,dataset/cifar/train/48309_deer.png,deer
3,dataset/cifar/train/33547_truck.png,truck
4,dataset/cifar/train/45202_automobile.png,automobile


In [12]:
test_df.head()

,name,class
0,dataset/cifar/test/2598_cat.png,cat
1,dataset/cifar/test/7195_horse.png,horse
2,dataset/cifar/test/6171_deer.png,deer
3,dataset/cifar/test/2586_horse.png,horse
4,dataset/cifar/test/8372_automobile.png,automobile


In [13]:
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    width_shift_range=0.3,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1./255.
)

In [22]:
train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)
test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='name',
    y_col='class',
    target_size=input_shape[:2],
    batch_size=batch_size
)

Found 50000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


## Build Model

In [16]:
inputs = layers.Input(input_shape)
net = layers.Conv2D(32, (3, 3), padding='SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64, (3, 3), padding='SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2, 2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs=inputs, outputs=net, name='Basic_CNN')

In [17]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),  # Optimization
              loss='categorical_crossentropy',  # Loss Function 
              metrics=['accuracy'])  # Metrics / Accuracy

## Training

In [19]:
batch_size

32

In [20]:
len(train_df)//32

1562

In [23]:
len(train_generator)

1563

In [24]:
model.fit_generator(
    train_generator,
    steps_per_epoch=len(train_getnerator),
    epochs=num_epochs,
    validation_data=test_generator,
    validation_steps=len(test_getnerator)
)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/10
1563/1563 [==============================] - 240s 154ms/step - loss: 1.8903 - accuracy: 0.2936 - val_loss: 1.6362 - val_accuracy: 0.4053
Epoch 2/10
1563/1563 [==============================] - 224s 143ms/step - loss: 1.6900 - accuracy: 0.3747 - val_loss: 1.6422 - val_accuracy: 0.3716
Epoch 3/10
1563/1563 [==============================] - 202s 129ms/step - loss: 1.6180 - accuracy: 0.4056 - val_loss: 1.6710 - val_accuracy: 0.3788
Epoch 4/10
1563/1563 [==============================] - 195s 125ms/step - loss: 1.5751 - accuracy: 0.4232 - val_loss: 1.5725 - val_accuracy: 0.4077
Epoch 5/10
1563/1563 [==============================] - 208s 133ms/step - loss: 1.5430 - accuracy: 0.4381 - val_loss: 1.5789 - val_accuracy: 0.4142
Epoch 6/10
1563/1563 [==============================] - 208s 133ms/step - loss: 1.5231 - accuracy: 0.4458 - val_loss: 1.6894 - val_accuracy: 0.3808
Epoch 7/10
1563/1563 [==============